# Data Modeling 

In [5]:
#libraries required
import pandas as pd 
import numpy as np
import os
import sys

In [8]:
#load dataset
file_path = "../data/processed/shot_events_processed.csv"

df= pd.read_csv(file_path)

print(print(f"Dataset has {df.shape[0]} rows and {df.shape[1]} columns"))

Dataset has 1494 rows and 9 columns
None


In [12]:
#confirm no missing values
print(df.isna().mean()*100)

#confirm data types
print(f"\n Data types in dataset \n{df.dtypes}")

shot_xg           0.0
location          0.0
body_part         0.0
shot_type         0.0
under_pressure    0.0
x                 0.0
y                 0.0
shot_distance     0.0
shot_angle        0.0
dtype: float64

 Data types in dataset 
shot_xg           float64
location           object
body_part          object
shot_type          object
under_pressure      int64
x                 float64
y                 float64
shot_distance     float64
shot_angle        float64
dtype: object


In [13]:
df.columns

Index(['shot_xg', 'location', 'body_part', 'shot_type', 'under_pressure', 'x',
       'y', 'shot_distance', 'shot_angle'],
      dtype='object')

In [17]:
#examine features: under_presusre, shot_type, body_part

#1. under_pressure
print(f"\n Unique values in 'under_pressure': \n{df['under_pressure'].unique()}")

#2. shot_type
print(f"\n Unique values in 'shot_type': \n{df['shot_type'].unique()}")

#3. body_part
print(f"\n Unique values in 'body_part' \n{df['body_part'].unique()}")


 Unique values in 'under_pressure': 
[0 1]

 Unique values in 'shot_type': 
['Open Play' 'Corner' 'Free Kick' 'Penalty']

 Unique values in 'body_part' 
['Right Foot' 'Left Foot' 'Head' 'Other']


In [18]:
#convert categorical features to category

#1. under_pressure
df['under_pressure'] = df['under_pressure'].astype('category')

#2. shot_type
df['shot_type'] = df['shot_type'].astype('category')

#3.body_part
df['body_part'] = df['body_part'].astype('category')

#validate changes 
print(f"\nData types for 'under_pressure','shot_type','body_part' : \n{df[['under_pressure','shot_type','body_part']].dtypes}")




Data types for 'under_pressure','shot_type','body_part' : 
under_pressure    category
shot_type         category
body_part         category
dtype: object
